In [21]:
!python --version

Python 3.10.11


In [3]:
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
from __future__ import annotations

import itertools
import sys
import pandas as pd
from stable_baselines3.common.logger import configure


In [3]:
import finrl
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.meta.preprocessor.preprocessors import data_split, FeatureEngineer,YahooDownloader
from finrl.plot import backtest_stats, get_baseline , plot_return

In [5]:
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2024-01-01'
PATH = '/files'
SYMBOLS = ['msft','ibm','hd','cat','amzn','intc','t','v','gs']
# INDICATORS = TECHNICAL_INDICATORS_LIST
RESULTS_DIR = '/results'
TIMESTEPS = 100000

In [6]:
date_col = "date"
tic_col = "tic"

In [8]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = SYMBOLS).fetch_data()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Shape of DataFrame:  (31698, 8)


In [47]:
df

,date,open,high,low,close,volume,tic,day
0,2010-01-04,6.812500,6.830500,6.657000,6.695000,151998000,amzn,0
1,2010-01-04,57.650002,59.189999,57.509998,40.336720,7325600,cat,0
2,2010-01-04,170.050003,174.250000,169.509995,135.576172,9135000,gs,0
3,2010-01-04,29.150000,29.150000,28.549999,20.428898,13120900,hd,0
4,2010-01-04,125.411087,127.122368,125.095604,76.767426,6438444,ibm,0
...,...,...,...,...,...,...,...,...
31693,2023-12-29,163.750000,164.179993,162.830002,162.072403,2525600,ibm,4
31694,2023-12-29,50.299999,50.570000,49.770000,50.103138,29266500,intc,4
31695,2023-12-29,376.000000,377.160004,373.480011,375.345886,18723000,msft,4
31696,2023-12-29,16.629999,16.830000,16.610001,16.244583,33271100,t,4


In [9]:
fe = FeatureEngineer(
        use_technical_indicator=True,
        tech_indicator_list=INDICATORS,
        use_vix=True,
        use_turbulence=True,
        user_defined_feature=False,
    )

In [9]:
processed = fe.preprocess_data(df)

Successfully added technical indicators


[*********************100%%**********************]  1 of 1 completed


Shape of DataFrame:  (3521, 8)
Successfully added vix
Successfully added turbulence index


In [10]:
dtypeProcessed = processed.dtypes
list_ticker = processed[tic_col].unique().tolist()
list_date = list(pd.date_range(processed[date_col].min(), processed[date_col].max()).astype(str))
combination = list(itertools.product(list_date, list_ticker))
final_data = pd.DataFrame(combination,columns = [date_col,tic_col]).merge(processed,how = 'left')
final_data = final_data[final_data[date_col].isin(processed[date_col])]
dtype_FinalDdata = final_data.dtypes
final_data = final_data.sort_values([date_col,tic_col])
final_data = final_data.fillna(0)
final_data.to_csv('Processed_data/'+'finaldata.csv')

In [ ]:
final_data = pd.read_csv('Processed_data/'+'finaldata.csv')

In [55]:
train_data = data_split(final_data,TRAIN_START_DATE,TRAIN_END_DATE)
trade_data = data_split(final_data, TRADE_START_DATE,TRADE_END_DATE)

In [ ]:
stock_dimension = len(train_data.tic.unique())
state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
buy_cost_list = sell_cost_list = [0.001]*stock_dimension
num_stock_shares = [0] * stock_dimension

In [19]:
initial_amount = 1000000
env_kwargs = {
    "hmax": 100,
    "initial_amount": initial_amount,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    }

In [ ]:
e_train_gym = StockTradingEnv(df=train_data, **env_kwargs)
env_train, _ = e_train_gym.get_sb_env()

In [13]:
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_sac = True
if_using_td3 = True

In [ ]:
agent = DRLAgent(env=env_train)

In [ ]:
if if_using_a2c:
        model_a2c = agent.get_model("a2c")
        # set up logger
        tmp_path = RESULTS_DIR + "/a2c"
        new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
        # Set new logger
        model_a2c.set_logger(new_logger_a2c)
        trained_a2c = agent.train_model(
            model=model_a2c, tb_log_name="a2c", total_timesteps=150000
        )
        trained_a2c.save('H:\TradingBot\save_models\\a2c.zip')


In [ ]:
if if_using_ddpg:
        model_ddpg = agent.get_model("ddpg")
        # set up logger
        tmp_path = RESULTS_DIR + "/ddpg"
        new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
        # Set new logger
        model_ddpg.set_logger(new_logger_ddpg)
        trained_ddpg = agent.train_model(
            model=model_ddpg, tb_log_name="ddpg", total_timesteps=150000
        )
        trained_ddpg.save('H:\TradingBot\save_models\ddpg.zip')

In [ ]:
if if_using_ppo:
        PPO_PARAMS = {
            "n_steps": 3072,
            "ent_coef": 0.01,
            "learning_rate": 0.00025,
            "batch_size": 128,
        }
        model_ppo = agent.get_model("ppo", model_kwargs=PPO_PARAMS)
        # set up logger
        tmp_path = RESULTS_DIR + "/ppo"
        new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
        # Set new logger
        model_ppo.set_logger(new_logger_ppo)
        trained_ppo = agent.train_model(
            model=model_ppo, tb_log_name="ppo", total_timesteps=150000
        )
        trained_ppo.save('H:\TradingBot\save_models\ppo.zip')

In [ ]:
if if_using_sac:
        SAC_PARAMS = {
            "batch_size": 128,
            "buffer_size": 100000,
            "learning_rate": 0.0001,
            "learning_starts": 100,
            "ent_coef": "auto_0.1",
        }
        model_sac = agent.get_model("sac", model_kwargs=SAC_PARAMS)
        # set up logger
        tmp_path = RESULTS_DIR + "/sac"
        new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
        # Set new logger
        model_sac.set_logger(new_logger_sac)
        trained_sac = agent.train_model(
            model=model_sac, tb_log_name="sac", total_timesteps=150000
        )
        
        trained_sac.save('H:\TradingBot\save_models\sac.zip')

In [ ]:
if if_using_td3:
        TD3_PARAMS = {"batch_size": 100, "buffer_size": 1000000, "learning_rate": 0.001}
        model_td3 = agent.get_model("td3", model_kwargs=TD3_PARAMS)
        # set up logger
        tmp_path = RESULTS_DIR + "/td3"
        new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
        # Set new logger
        model_td3.set_logger(new_logger_td3)
        trained_td3 = agent.train_model(
            model=model_td3, tb_log_name="td3", total_timesteps=150000
        )
        
        trained_td3.save('H:\TradingBot\save_models\\td3.zip')